In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [ ]:
#Must be saved out by running the code in DAN_MBON_paths_analysis.ipynb

W = np.load("W.npy")


In [ ]:
unique_neurons = list(np.load("unique_neurons.npy"))

In [ ]:
sources = [unique_neurons.index(idx) for idx in np.load("mbon_ids.npy")]
targets = [unique_neurons.index(idx) for idx in np.load("dan_ids.npy")]


In [ ]:
#Avoid paths that loop back to source nodes
for s in sources:
    W[:, s] = 0
    
#for t in targets:
#    W[t, :] = 0

In [ ]:
#Scaling of most computationally intensive matrix multiplication step in the vectorized centrality algorithm



N_options = [2**10, 2**11, 2**12, 2**13, 2**14] #Subsampled graph sizes
cpu_times = np.zeros([len(N_options)])
cpu_noparallel_times = np.zeros([len(N_options)])
gpu_times = np.zeros([len(N_options)])

for mode in ["gpu", "cpu", "cpu_noparallel"]:
    for N in N_options:
        W_sub = W[:N, :N] #Subsampling step to test scaling with graph size
        print(mode, N)
    
        if mode == "gpu":
            W_cuda = torch.Tensor(W_sub).cuda()
        elif mode == "cpu":
            W_cuda = torch.Tensor(W_sub).cpu()
        elif mode == "cpu_noparallel":
            W_cuda = np.array(W_sub)
            
        

        start_time = time.monotonic()
        k_max = 9 #max path length
        W_effective_list = [W_sub]
        current = W_cuda
        if mode == "cpu_noparallel":
            current = np.copy(W_sub)
        for k in range(k_max):
            if mode == "cpu_noparallel":
                current = np.matmul(current, W_sub)
                W_effective_list.append(np.copy(current))
            else:
                current = torch.matmul(current, W_cuda)
                W_effective_list.append(current.cpu().detach().numpy())

        end_time = time.monotonic()

        total_time = end_time - start_time

        if mode == "gpu":
            gpu_times[N_options.index(N)] = total_time
        elif mode == "cpu":
            cpu_times[N_options.index(N)] = total_time
        elif mode == "cpu_noparallel":
            cpu_noparallel_times[N_options.index(N)] = total_time

In [ ]:
#Plot scaling analysis

plt.plot(np.log2(gpu_times))
plt.plot(np.log2(cpu_times))
plt.plot(np.log2(cpu_noparallel_times))
yticks = np.arange(-4, 7, 2)
plt.yticks(yticks, np.power(2.0, yticks), fontsize=14)
plt.xticks(range(len(N_options)), N_options, fontsize=14)

plt.legend(["GPU", "CPU (parallel)", "CPU (serial)"], fontsize=14)

plt.ylabel("Running time (s)", fontsize=16)
plt.xlabel("Graph size (N)", fontsize=16)
plt.tight_layout()
plt.savefig("Matrix_alg_scaling.pdf")
plt.show()

In [ ]:
#Run vectorized centrality algorithm on full data

W_cuda = torch.Tensor(W).cuda()

k_max = 9
W_effective_list = [W]
current = W_cuda
for k in range(k_max):
    print(k)
    current = torch.matmul(current, W_cuda)
    W_effective_list.append(current.cpu().detach().numpy())
    
    


In [ ]:
#compute (c^k_n)^(in) and (c^k_n)^(out)

contribution_to_targets_list = []
contribution_from_sources_list = []
for k in range(k_max):
    contribution_to_targets = W_effective_list[k][:, targets].sum(1)
    contribution_from_sources = W_effective_list[k][sources].sum(0)
    
    contribution_to_targets_list.append(contribution_to_targets)
    contribution_from_sources_list.append(contribution_from_sources)
    
#compute (c^k_n)
contribution_list = []
for k in range(3, k_max):
    contribution = np.zeros([W.shape[0]])
    for k_in in range(1, k-1):
        contribution += (contribution_from_sources_list[k_in] * contribution_to_targets_list[k-k_in]) / k
    contribution_list.append(contribution)
    

In [ ]:
#Summary statistic: total contribution of paths of different lengths

totals = []
for k in range(k_max):
    totals.append(W_effective_list[k][sources][:, targets].sum())
for k in range(k_max):
    plt.bar(k, totals[k] / np.sum(totals), color="tab:blue")
    
plt.xticks(range(k_max), np.arange(k_max)+1)
plt.xlabel("Number of steps", fontsize=16)
plt.ylabel("Fractional contribution\nto interaction", fontsize=16)

plt.tight_layout()
plt.savefig("Matrix_alg_steps_contribution.pdf")
plt.show()

In [ ]:
unique_neurons_types = np.load("unique_neurons_types.npy") 

In [ ]:
#Code for aggregating contribution of different cell types to paths of different lengths k between sources and targets


type_count = {}
for k in range(k_max):
    type_count[k] = {}
type_count_overall = {}
for k in range(k_max-3):
    for n in range(len(unique_neurons_types)):
        typ = unique_neurons_types[n]
        if typ is None:
            continue
        coarse_typ = ""
        for t in typ:
            if t.isalpha() and t.isupper():
                coarse_typ = coarse_typ + t
        if coarse_typ not in type_count[k+3].keys():
            type_count[k+3][coarse_typ] = 0
            
        if coarse_typ not in type_count_overall.keys():
            type_count_overall[coarse_typ] = 0
            
        
        
        type_count[k+3][coarse_typ] += contribution_list[k][n]
        type_count_overall[coarse_typ] += contribution_list[k][n]

In [ ]:
#Plot centrality analysis

import pandas as pd
from collections import Counter, OrderedDict

k = 3
counts = type_count_overall
df = pd.DataFrame.from_dict(counts, orient='index')
df.sort_values(0, ascending=False, inplace=True)
df[0] = df[0].values / np.sum(df[0].values)
df = df[:20]
fig = plt.figure(figsize=(6, 5))
df.plot(kind='bar', figsize=(10, 5), legend=False)

plt.ylabel("Centrality", fontsize=16)
plt.xlabel("Cell type", fontsize=16)
plt.tight_layout()

plt.savefig("Centrality_fig_matrix_alg.pdf")
plt.show()
